In [7]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *


import os,sys
from importlib import reload
import copy
sys.path.append("../tracker")

In [19]:
import kalmanfilter as KF
import utilities as Util
import trackfinder as TF
import datatypes

reload(KF)
reload(KU)

<module 'utilities' from '../tracker\\utilities.py'>

In [9]:
# Generated toy Events

N_events=500
Tracks_per_event = 4
N_tracks = N_events * Tracks_per_event
Hit_collection={}
Truth_collection ={}
ParTruth_collection={}

x0s = np.random.uniform(-100,100,N_tracks)
z0s = np.random.uniform(-100,100,N_tracks)
t0s = np.random.uniform(-3,3,N_tracks)
Axs = np.random.uniform(-0.3,0.3,N_tracks)
Azs = np.random.uniform(-0.3,0.3,N_tracks)
Ats = np.random.uniform(1/28,1/32,N_tracks)

for i in range(N_events):
    # Make multiple tracks per event    
    tracks, truths, partruth = [],[],[]
    for j in range(Tracks_per_event):
        k =Tracks_per_event*i+j
        a,b,c = Util.hit.gen_hits(x0=x0s[k],y0=0,z0=z0s[k], t0=t0s[k], Ax=Axs[k],Az=Azs[k],At=Ats[k],  N_LAYERS=4)
        tracks.extend(a)
        truths.extend(b)
        partruth.extend(c)

    Hit_collection[i],Truth_collection[i],ParTruth_collection[i] = tracks, truths, partruth




### Confirm that chi2_predict equals to chi2_filtered

In [10]:
hits = Hit_collection[4][:4]
n_hits = len(hits)

kf_find = KF.KalmanFilterFind()

# Set initial state using first two hits
m0, V0, H0, Xf0, Cf0, Rf0 = Util.track.init_state(hits)
kf_find.init_filter( m0, V0, H0, Xf0, Cf0, Rf0)


# Feed all measurements to KF
for i in range(2,n_hits):   
    # some processing
    hit = hits[i]
    dy  = hits[i].y-hits[i-1].y
    mi, Vi, Hi, Fi, Qi = Util.track.add_measurement(hit, dy)
    
    # pass to KF
    kf_find.update_matrix(Vi, Hi, Fi, Qi)
    chi2_predict = kf_find.forward_predict_chi2(mi)
    chi2_filtered = kf_find.forward_filter(mi)
    print("chi2 chi2_predict, chi2_filtered", chi2_predict, chi2_filtered)


kf = Util.track.run_kf(hits)
print(kf.chift)

print(kf.Xf[-1])
print(kf_find.Xf)


chi2 chi2_predict, chi2_filtered 10.058218638455461 10.05821863845534
chi2 chi2_predict, chi2_filtered 2.9392526633601803 2.9392526633601643
[0, 10.05821863845534, 2.9392526633601643]
[ 6.30251528e+01  2.26868016e+01  8.85132713e+00  1.11558917e-01
 -2.81000088e-01  3.90167943e-02]
[ 6.30251528e+01  2.26868016e+01  8.85132713e+00  1.11558917e-01
 -2.81000088e-01  3.90167943e-02]


## Track finder algorithm Outline


Pseudo code:

```python
hits # provide a list of hits
seeds = seed(hts) # Make a list of seed
while len(seeds)>0:
    seed = seeds[0]

    # Round 1: find hits
    hits_found = find()
    seed.pop(0) # Remove the used seed
    continue if len(hits_found)<nhits_min

    # Round 2: run KF and drop outlier hits during smoothing
    hits_found = run_kf_drop()
    continue if len(hits_found)<nhits_min

    # Round 3: run KF (no dropping)
    run_kf(drop=False)
    remove_related_hits_seeds()

```

In [25]:
# Test that it can find one track correctly

hits = Hit_collection[0]
hits_grouped = Util.track.group_hits_by_layer(hits)

tf = TF.TrackFinder(method="recursive")
# tf = TrackFinder(method="gredy")
seeds = tf.seeding(hits)
# seed=seeds[0]
seed=[0,10]
hits_found = tf.find_once(hits, hits_grouped, seed)
# [print(hit) for hit in hits_found]
hits_found


([Hit(x=48.31525830236538, y=1200, z=-103.5, t=-3.130196552885981, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=0, ind=0),
  Hit(x=72.0, y=1280, z=-97.76614637380844, t=1.5038908084352176, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=1, ind=1),
  Hit(x=90.55200518754091, y=1360, z=54.0, t=1.6323063768840542, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=2, ind=10),
  Hit(x=94.5, y=1440, z=53.93844211089895, t=5.6278145722667725, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=3, ind=11)],
 46.98224698773643)

In [26]:
reload(TF)

<module 'trackfinder' from '../tracker\\trackfinder.py'>

In [27]:
tf = TF.TrackFinder(method="recursive")
tf.find(hits)
print(tf.tracks_found)
tf.run(hits)
    

[[Hit(x=79.28172441833514, y=1200, z=85.5, t=-3.0570367421714155, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=0, ind=8), Hit(x=81.0, y=1280, z=57.41893115341553, t=2.158136592510161, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=1, ind=9), Hit(x=85.73832269621649, y=1360, z=58.5, t=7.118180428129357, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=2, ind=14), Hit(x=94.5, y=1440, z=39.831227996637566, t=11.47726642449396, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=3, ind=15)], [Hit(x=41.699158998584146, y=1200, z=85.5, t=3.251538012275695, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=0, ind=12), Hit(x=67.5, y=1280, z=61.51479782440997, t=6.437425323605671, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=1, ind=13), Hit(x=90.55200518754091, y=1360, z=54.0, t=1.6323063768840542, x_err=14.132352000025548, y_err=0, z_

[Track(x0=74.34571271707965, y0=1200, z0=85.45987979792076, t=-2.8603063246017815, Ax=0.08385368396556737, Ay=1, Az=-0.16927827129782047, At=0.060703691669519146, cov=array([[ 4.12807664e+00,  0.00000000e+00,  0.00000000e+00,
         -2.06577639e-02,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.68043024e+00,  0.00000000e+00,
          0.00000000e+00, -1.04592372e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  7.00000000e-01,
          0.00000000e+00,  0.00000000e+00, -3.75000000e-03],
        [-2.06577639e-02,  0.00000000e+00,  0.00000000e+00,
          1.29654171e-04,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.04592372e-02,  0.00000000e+00,
          0.00000000e+00,  1.29654171e-04,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -3.75000000e-03,
          0.00000000e+00,  0.00000000e+00,  3.12500000e-05]]), chi2=1.5218850516073403, ind=0, hits=[8, 9, 14, 15], hits_filtered=[[74.34571271707965, 1200, 85.459879797

In [28]:
Truth_collection[0]

[Hit(x=55.73232151867401, y=1200, z=-99.0941029448389, t=-2.1376683808388135, x_err=0, y_err=0, z_err=0, t_err=0, layer=0, ind=0),
 Hit(x=75.52207100124639, y=1280, z=-109.90535966969796, t=0.6099907995563818, x_err=0, y_err=0, z_err=0, t_err=0, layer=1, ind=1),
 Hit(x=95.31182048381875, y=1360, z=-120.71661639455702, t=3.357649979951577, x_err=0, y_err=0, z_err=0, t_err=0, layer=2, ind=2),
 Hit(x=115.10156996639111, y=1440, z=-131.52787311941609, t=6.105309160346773, x_err=0, y_err=0, z_err=0, t_err=0, layer=3, ind=3),
 Hit(x=-71.26644688341386, y=1200, z=34.1021687782565, t=1.7687581426746357, x_err=0, y_err=0, z_err=0, t_err=0, layer=0, ind=0),
 Hit(x=-74.46866476715925, y=1280, z=37.13813365225473, t=4.3101286405402846, x_err=0, y_err=0, z_err=0, t_err=0, layer=1, ind=1),
 Hit(x=-77.67088265090464, y=1360, z=40.174098526252955, t=6.851499138405934, x_err=0, y_err=0, z_err=0, t_err=0, layer=2, ind=2),
 Hit(x=-80.87310053465004, y=1440, z=43.21006340025119, t=9.392869636271584, x_err

## Run on an event that failed in the tracker

In [29]:
x = [241.62934581783023, 259.25, 221.49664808213277, 227.75]
y = [9894.0, 9975.599999999999, 10057.2, 10138.8]
z = [12456.75, 12450.5, 12461.25, 12450.5]
t = [40.91365706617539, 45.79668867914941, 48.189471653643565, 50.295569678622876]
layers = [0,1,2,3,]

hits = Util.hit.make_hits(x,y,z,t,layers)

tf = TF.TrackFinder(method="recursive")
# tf.find(hits)
# print(tf.tracks_found)
tf.run(hits)

[Track(x0=274.2196119526758, y0=9894.0, z0=12456.768104297438, t=41.71806864762234, Ax=-0.18964018097796365, Ay=1, Az=0.026258905960041642, At=0.0374246578576429, cov=array([[ 4.12807664e+00,  0.00000000e+00,  0.00000000e+00,
         -2.02527097e-02,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.68043024e+00,  0.00000000e+00,
          0.00000000e+00, -1.02541541e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  7.00000000e-01,
          0.00000000e+00,  0.00000000e+00, -3.67647059e-03],
        [-2.02527097e-02,  0.00000000e+00,  0.00000000e+00,
          1.24619542e-04,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.02541541e-02,  0.00000000e+00,
          0.00000000e+00,  1.24619542e-04,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -3.67647059e-03,
          0.00000000e+00,  0.00000000e+00,  3.00365244e-05]]), chi2=11.199094002959015, ind=0, hits=[0, 1, 2, 3], hits_filtered=[[274.2196119526758, 9894.0, 12456.768104

In [30]:
import joblib
joblib.dump(hits, "hits_example.joblib")

['hits_example.joblib']